In [1]:
%load_ext autoreload
%autoreload 2


In [8]:
from datasets import load_from_disk
from transformers import (
    RobertaTokenizer,
    T5ForConditionalGeneration,
)
import torch
import random
import json
from draw_svg import *


In [3]:
torch.set_num_threads(16)

In [4]:
tokenizer = RobertaTokenizer.from_pretrained("Salesforce/codet5-small")
model = T5ForConditionalGeneration.from_pretrained(
    "/data/nicolasmaier/model/codet5-finetuned-seq-4/checkpoint-27000"
)


In [5]:
device = 'cpu'# 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(device)
model_gpu = model.to(device)

cpu


In [6]:
dataset = load_from_disk("/data/nicolasmaier/dataset/hf_clean_seq_dataset_2")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['code', 'contents', 'xmi', 'originalLine', 'input_ids', 'attention_mask', 'seq', 'labels'],
        num_rows: 381183
    })
    valid: Dataset({
        features: ['code', 'contents', 'xmi', 'originalLine', 'input_ids', 'attention_mask', 'seq', 'labels'],
        num_rows: 13383
    })
    test: Dataset({
        features: ['code', 'contents', 'xmi', 'originalLine', 'input_ids', 'attention_mask', 'seq', 'labels'],
        num_rows: 22333
    })
})


In [15]:
example_orig = dataset["test"][6000]
print(example_orig["originalLine"])

print(tokenizer.decode(example_orig["input_ids"]))
print()
print(tokenizer.decode(example_orig["labels"]))
print()

input = tokenizer(example_orig["contents"], return_tensors="pt").to(device)

outputs = model_gpu.generate(
    input.input_ids,
    num_beams=10,
    max_length=510,
    # do_sample=True,
    temperature=0.3,
    top_k=50,
    top_p=0.8,
)
model_output = tokenizer.decode(outputs[0][2:-1])
print(model_output)

res = None
while res is None:
    try:
        res = json.loads(model_output)
    except:
        print("problem")
        model_output = model_output[:-1]

print(res)

with open("diff_out_1.json", "w") as f:
    json.dump(json.loads(tokenizer.decode(example_orig["labels"][1:-1])), f, indent=2)
with open("diff_out_2.json", "w") as f:
    json.dump(res, f, indent=2)


6830
<s>public static JBossPortComponentMetaData getJBossWebserviceMetaDataPortComponent(
        final DeploymentUnit unit, final String name) {

        if (name!= null) {
            final JBossWebservicesMetaData jbossWebserviceMetaData = unit.getAttachment(JBOSS_WEBSERVICES_METADATA_KEY);

            if (jbossWebserviceMetaData!= null) {
                JBossPortComponentMetaData[] portComponent = jbossWebserviceMetaData.getPortComponents();

                if (portComponent!= null) {
                    for (JBossPortComponentMetaData component : portComponent) {
                        if (name.equals(component.getEjbName())) {
                            return component;
                        }
                    }
                }
            }
        }
        return null;
    }</s>

<s>[{"type": "blocks", "blocks": [{"name": "if", "guard": "name!= null", "contents": [{"type": "methodInvocation", "to": ["unit"], "method": "getAttachment"}, {"type": "scopedVariable", "

In [18]:
seq_data = json.loads(tokenizer.decode(dataset["test"][4000]["labels"][1:-1]))
print(seq_data)

[{'type': 'blocks', 'blocks': [{'name': 'if', 'guard': 'mv!= null', 'contents': [{'type': 'methodInvocation', 'to': ['mv'], 'method': 'visitAnnotation'}, {'type': 'return', 'value': '[...].visitAnnotation([...])'}]}]}, {'type': 'return', 'value': 'null'}]


In [22]:
print(draw_sequence_diagram(seq_data))

[{'type': 'blocks', 'blocks': [{'name': 'if', 'guard': 'mv!= null', 'contents': [{'type': 'methodInvocation', 'to': ['mv'], 'method': 'visitAnnotation'}, {'type': 'return', 'value': '[...].visitAnnotation([...])'}]}]}, {'type': 'return', 'value': 'null'}]
=====

